# **07. 피봇테이블과 그룹분석**

## **목차**

1. 피봇테이블  
2. 그룹분석  
    2.1 groupby 메서드  
    2.2 그룹연산 메서드  
        연습문제 7-1 
        연습문제 7-2
        
3. pivot_table  
    3.1 TIP 데이터 예제  
    3.2 그룹별 통계  
        연습문제 7-3 
        연습문제 7-4 

## **1. 피봇테이블**

피봇테이블(pivot table)이란 데이터 열 중에서 두 개의 열을 각각 행 인덱스, 열 인덱스로 사용하여 데이터를 조회하여 펼쳐놓은 것을 말한다.

Pandas는 피봇테이블을 만들기 위한  `pivot` 메서드를 제공한다. 첫번째 인수로는 행 인덱스로 사용할 열 이름, 두번째 인수로는 열 인덱스로 사용할 열 이름, 그리고 마지막으로 데이터로 사용할 열 이름을 넣는다.

Pandas는 지정된 두 열을 각각 행 인덱스와 열 인덱스로 바꾼 후 행 인덱스의 라벨 값이 첫번째 키의 값과 같고 열 인덱스의 라벨 값이 두번째 키의 값과 같은 데이터를 찾아서 해당 칸에 넣는다. 만약 주어진 데이터가 존재하지 않으면 해당 칸에 `NaN` 값을 넣는다.

다음 데이터는 각 도시의 연도별 인구를 나타낸 것이다.

In [2]:
import pandas as pd

data = {
    "도시": ["서울", "서울", "서울", "부산", "부산", "부산", "인천", "인천"],
    "연도": ["2015", "2010", "2005", "2015", "2010", "2005", "2015", "2010"],
    "인구": [9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 263203],
    "지역": ["수도권", "수도권", "수도권", "경상권", "경상권", "경상권", "수도권", "수도권"]
}
columns = ["도시", "연도", "인구", "지역"]
df1 = pd.DataFrame(data, columns=columns)
df1

도시    연도       인구   지역
0  서울  2015  9904312  수도권
1  서울  2010  9631482  수도권
2  서울  2005  9762546  수도권
3  부산  2015  3448737  경상권
4  부산  2010  3393191  경상권
5  부산  2005  3512547  경상권
6  인천  2015  2890451  수도권
7  인천  2010   263203  수도권

이 데이터를 도시 이름이 행 인덱스가 되고 연도가 열 인덱스가 되어 행과 열 인덱스만 보면 어떤 도시의 어떤 시점의 인구를 쉽게 알 수 있도록 피봇테이블로 만들어보자. `pivot` 명령으로 사용하고 행 인덱스 인수로는 `"도시"`, 열 인덱스 인수로는 `"연도"`, 데이터 이름으로 `"인구"`를 입력하면 된다.

In [3]:
# pivot(행, 열, 데이터)

df1.pivot("도시", "연도", "인구")  # 행 : 도시, 열 : 연도, 데이터이름 : 인구 

연도       2005       2010       2015
도시                                 
부산  3512547.0  3393191.0  3448737.0
서울  9762546.0  9631482.0  9904312.0
인천        NaN   263203.0  2890451.0

이 피봇테이블의 값 3512547은 "도시"가 부산이고 "연도"가 2005년인 데이터를 "인구"열에서 찾은 값이다. 2005년 인천의 인구는 데이터에 없기 때문에 `NaN`으로 표시된다.

사실 피봇테이블은 다음과 같이 `set_index` 명령과 `unstack` 명령을 사용해서 만들 수도 있다.

In [4]:
# set_index([행, 열])[[데이터]].unstack()

df1.set_index(["도시", "연도"])[["인구"]].unstack()

인구                      
연도       2005       2010       2015
도시                                 
부산  3512547.0  3393191.0  3448737.0
서울  9762546.0  9631482.0  9904312.0
인천        NaN   263203.0  2890451.0

행 인덱스와 열 인덱스는 **데이터를 찾는 키(key)**의 역할을 한다. 따라서 키 값으로 데이터가 **단 하나만 찾아져야 한다.**
만약 행 인덱스와 열 인덱스 조건을 만족하는 데이터가 2개 이상인 경우에는 에러가 발생한다. 예를 들어 위 데이터프레임에서 ("지역", "연도")를 키로 하면 ("수도권", "2015")에 해당하는 값이 두 개 이상이므로 다음과 같이 에러가 발생한다. 

In [4]:
try:
    df1.pivot("지역", "연도", "인구")
except ValueError as e:
    print("Error:", e)

Error: Index contains duplicate entries, cannot reshape


## **2. 그룹분석**

만약 키가 지정하는 조건에 맞는 데이터가 하나 이상이라서 데이터 그룹을 이루는 경우에는 그룹의 특성을 보여주는 **그룹분석(group analysis)**을 한다.

그룹분석은 피봇테이블과 달리 키에 의해서 결정되는 데이터가 여러개가 있을 경우 미리 지정한 연산을 통해 그 그룹 데이터의 대표값을 계산한. Pandas에서는 `groupby` 명령을 사용하여 다음처럼 그룹분석을 한다.

1. 분석하고자 하는 시리즈나 데이터프레임에 `groupby` 메서드를 호출하여 그룹화를 한다.
1. 그룹 객체에 대해 그룹연산을 수행한다.

### **2.1 `groupby` 메서드**

`groupby` 메서드는 데이터를 그룹 별로 분류하는 역할을 한다. `groupby` 메서드의 인수로는 다음과 같은 값을 사용한다.

 * 열 또는 열의 리스트 
 * 행 인덱스

연산 결과로 그룹 데이터를 나타내는 `GroupBy` 클래스 객체를 반환한다. 이 객체에는 그룹별로 연산을 할 수 있는 그룹연산 메서드가 있다.

### **2.2 그룹연산 메서드**

`groupby` 결과, 즉 `GroupBy` 클래스 객체의 뒤에 붙일 수 있는 그룹연산 메서드는 다양하다. 다음은 자주 사용되는 그룹연산 메서드들이다.

* `size`, `count`: 그룹 데이터의 갯수
* `mean`, `median`, `min`, `max`: 그룹 데이터의 평균, 중앙값, 최소, 최대
* `sum`, `prod`, `std`, `var`, `quantile` : 그룹 데이터의 합계, 곱, 표준편차, 분산, 사분위수
* `first`, `last`: 그룹 데이터 중 가장 첫번째 데이터와 가장 나중 데이터
   
이 외에도 많이 사용되는 것으로는 다음과 같은 그룹연산이 있다.

* `agg`, `aggregate`
    * 만약 원하는 그룹연산이 없는 경우 함수를 만들고 이 함수를 `agg`에 전달한다.
    * 또는 여러가지 그룹연산을 동시에 하고 싶은 경우 함수 이름 문자열의 리스트를 전달한다.

* `describe`
    * 하나의 그룹 대표값이 아니라 여러개의 값을 데이터프레임으로 구한다.
   
* `apply`
    *  `describe` 처럼 하나의 대표값이 아닌 데이터프레임을 출력하지만 원하는 그룹연산이 없는 경우에 사용한다.
    
* `transform`
    * 그룹에 대한 대표값을 만드는 것이 아니라 그룹별 계산을 통해 데이터 자체를 변형한다.

예를 들어 다음과 같은 데이터가 있을 때 key1의 값(A 또는 B)에 따른 data1의 평균은 어떻게 구할까?

In [5]:
np.random.seed(0)
df2 = pd.DataFrame({
    'key1': ['A', 'A', 'B', 'B', 'A'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': [1, 2, 3, 4, 5],
    'data2': [10, 20, 30, 40, 50]
})
df2

key1 key2  data1  data2
0    A  one      1     10
1    A  two      2     20
2    B  one      3     30
3    B  two      4     40
4    A  one      5     50

`groupby` 명령을 사용하여 그룹 A와 그룹 B로 구분한 그룹 데이터를 만든다.

In [6]:
groups = df2.groupby(df2.key1)
groups

이 `GroupBy` 클래스 객체에는 각 그룹 데이터의 인덱스를 저장한 `groups` 속성이 있다.

In [7]:
groups.groups

{'A': Int64Index([0, 1, 4], dtype='int64'),
 'B': Int64Index([2, 3], dtype='int64')}

A그룹과 B그룹 데이터의 합계를 구하기 위해 `sum`이라는 그룹연산을 한다.

In [8]:
groups.sum()

data1  data2
key1              
A         8     80
B         7     70

`GroupBy` 클래스 객체를 명시적으로 얻을 필요가 없다면 `groupby` 메서드와 그룹연산 메서드를 연속으로 호출한다. 다음 예제는 열 `data1`에 대해서만 그룹연산을 하는 코드이다.

In [9]:
df2.data1.groupby(df2.key1).sum()

key1
A    8
B    7
Name: data1, dtype: int64

데이터를 그룹으로 나눈 `GroupBy` 클래스 객체 또는 그룹분석한 결과에서 `data1`만 뽑아도 된다.

In [10]:
df2.groupby(df2.key1)["data1"].sum()   # `GroupBy` 클래스 객체에서 data1만 선택하여 분석하는 경우

key1
A    8
B    7
Name: data1, dtype: int64

In [11]:
df2.groupby(df2.key1).sum()["data1"]  # 전체 데이터를 분석한 후 data1만 선택한 경우

key1
A    8
B    7
Name: data1, dtype: int64

### **연습 문제 7-1**

key1의 값을 기준으로 data1의 값을 분류하여 합계를 구한 결과를 시리즈가 아닌 데이터프레임으로 나타내라.


In [6]:
import numpy as np
import pandas as pd

np.random.seed(0)
df2 = pd.DataFrame({
    'key1': ['A', 'A', 'B', 'B', 'A'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': [1, 2, 3, 4, 5],
    'data2': [10, 20, 30, 40, 50]
})
df2

key1 key2  data1  data2
0    A  one      1     10
1    A  two      2     20
2    B  one      3     30
3    B  two      4     40
4    A  one      5     50

In [24]:

to_df = pd.DataFrame(df2.data1.groupby(df2.key1).sum())
print(type(to_df))
to_df

<class 'pandas.core.frame.DataFrame'>


data1
key1       
A         8
B         7

<hr>

이번에는 복합 키 (key1, key2) 값에 따른 data1의 합계를 구하자. 
**분석하고자 하는 키가 복수이면 리스트를 사용한다.**

In [25]:
df2.data1.groupby([df2.key1, df2.key2]).sum()  # 복합 키 사용하기 

key1  key2
A     one     6
      two     2
B     one     3
      two     4
Name: data1, dtype: int64

이 결과를 `unstack` 명령으로 피봇 데이블 형태로 만들수도 있다.

In [26]:
df2.data1.groupby([df2["key1"], df2["key2"]]).sum().unstack("key2")  # Key2를 쌓지않는다. 즉 열 이름으로 뺀다.

key2  one  two
key1          
A       6    2
B       3    4

그룹분석 기능을 사용하면 위의 인구 데이터로부터 지역별 합계를 구할 수도 있다.

In [31]:
# 지역별(행) 연도별(열) 인구 

df1["인구"].groupby([df1["지역"], df1["연도"]]).sum().unstack("연도")  # '연도'를 열 이르믕로 쓰는 피봇테이블 

연도      2005     2010      2015
지역                             
경상권  3512547  3393191   3448737
수도권  9762546  9894685  12794763

다음 데이터는 150 송이의 붓꽃(iris)에 대해 붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎폭(sepal_width), 꽃잎폭(sepal_width), 꽃잎폭(sepal_width)을 측정한 데이터이다. (Seaborn 패키지가 설치되어 있어야 한다. 

In [33]:
import seaborn as sns
iris = sns.load_dataset("iris")

각 붓꽃 종별로 가장 큰 값과 가장 작은 값의 비율을 구해보자. 이러한 계산을 하는 그룹연산 메서드는 없으므로 직접 만든 후 `agg` 메서드를 적용한다.

In [34]:
def peak_to_peak_ratio(x):
    return x.max() / x.min()

iris.groupby(iris.species).agg(peak_to_peak_ratio)

sepal_length  sepal_width  petal_length  petal_width
species                                                         
setosa          1.348837     1.913043      1.900000     6.000000
versicolor      1.428571     1.700000      1.700000     1.800000
virginica       1.612245     1.727273      1.533333     1.785714

`describe` 메서드를 사용하면 다양한 기술 통계(descriptive statistics)값을 한 번에 구한다. 그룹별로 하나의 스칼라 값이 아니라 하나의 데이터프레임이 생성된다는 점에 주의하라.

In [35]:
# 종별 속성별 기술통계량 

iris.groupby(iris.species).describe().T

species                setosa  versicolor  virginica
sepal_length count  50.000000   50.000000  50.000000
             mean    5.006000    5.936000   6.588000
             std     0.352490    0.516171   0.635880
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.225000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
sepal_width  count  50.000000   50.000000  50.000000
             mean    3.428000    2.770000   2.974000
             std     0.379064    0.313798   0.322497
             min     2.300000    2.000000   2.200000
             25%     3.200000    2.525000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.675000    3.000000   3.175000
             max     4.400000    3.400000   3.800000
petal_length count  50.000000   50.000000  50.000000
             mean    1.462000    4.260000   5.552000
             std     0.173664    0.469911   0.551895
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.350000   5.550000
             75%     1.575000    4.600000   5.875000
             max     1.900000    5.100000   6.900000
petal_width  count  50.000000   50.000000  50.000000
             mean    0.246000    1.326000   2.026000
             std     0.105386    0.197753   0.274650
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000

`apply` 메서드를 사용하면 `describe` 메서드처럼 하나의 그룹에 대해 하나의 대표값(스칼라 값)을 구하는 게 아니라 데이터프레임을 만들 수 있다. 예를 들어 다음처럼 각 붓꽃 종별로 가장 꽃잎 길이(petal length)가 큰 3개의 데이터를 뽑아낼 수도 있다.

In [18]:
def top3_petal_length(df):
    return df.sort_values(by="petal_length", ascending=False)[:3]

iris.groupby(iris.species).apply(top3_petal_length)

sepal_length  sepal_width  petal_length  petal_width  \
species                                                                
setosa     24            4.8          3.4           1.9          0.2   
           44            5.1          3.8           1.9          0.4   
           23            5.1          3.3           1.7          0.5   
versicolor 83            6.0          2.7           5.1          1.6   
           77            6.7          3.0           5.0          1.7   
           72            6.3          2.5           4.9          1.5   
virginica  118           7.7          2.6           6.9          2.3   
           117           7.7          3.8           6.7          2.2   
           122           7.7          2.8           6.7          2.0   

                   species  
species                     
setosa     24       setosa  
           44       setosa  
           23       setosa  
versicolor 83   versicolor  
           77   versicolor  
           72   versicolor  
virginica  118   virginica  
           117   virginica  
           122   virginica

`transform` 메서드는 그룹별 대표값을 만드는 것이 아니라 **그룹별 계산을 통해 데이터프레임 자체를 변화**시킨다. 따라서 만들어진 데이터프레임의 크기는 원래 데이터프레임과 같다. 예를 들어 다음처럼 각 붓꽃이 해당 종 중에서 꽃잎길이가 대/중/소 어느 것에 해당되는지에 대한 데이터프레임을 만들 수도 있다.

In [19]:
def q3cut(s):
    return pd.qcut(s, 3, labels=["소", "중", "대"])


iris["petal_length_class"] = iris.groupby(iris.species)["petal_length"].transform(q3cut)
iris[["petal_length", "petal_length_class"]].tail(10)

petal_length petal_length_class
140           5.6                  중
141           5.1                  소
142           5.1                  소
143           5.9                  대
144           5.7                  중
145           5.2                  소
146           5.0                  소
147           5.2                  소
148           5.4                  중
149           5.1                  소

### **연습 문제 7-2**

붓꽃(iris) 데이터에서 붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎폭(sepal_width) 등의 평균을 구하라.
만약 붓꽃 종(species)이 표시되지 않았을 때 이 수치들을 이용하여 붓꽃 종을 찾아낼 수 있을지 생각하라.

In [171]:
import seaborn as sns
iris = sns.load_dataset("iris")

iris.groupby(iris.species).mean().T

species       setosa  versicolor  virginica
sepal_length   5.006       5.936      6.588
sepal_width    3.428       2.770      2.974
petal_length   1.462       4.260      5.552
petal_width    0.246       1.326      2.026

## **3. `pivot_table`**

Pandas는 `pivot` 명령과 `groupby` 명령의 중간 성격을 가지는 `pivot_table` 명령도 제공한다.

`pivot_table` 명령은 `groupby` 명령처럼 그룹분석을 하지만 최종적으로는 `pivot` 명령처럼 피봇테이블을 만든다. 즉 **`groupby` 명령의 결과에 `unstack`을 자동 적용**하여 2차원적인 형태로 변형한다. 사용 방법은 다음과 같다.

* `pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, margins_name='All')`
    * `data`: 분석할 데이터프레임 (메서드일 때는 필요하지 않음)
    * `values`: 분석할 데이터프레임에서 분석할 열
    * `index`: 행 인덱스로 들어갈 키 열 또는 키 열의 리스트
    * `columns`: 열 인덱스로 들어갈 키 열 또는 키 열의 리스트
    * `aggfunc`: 분석 메서드
    * `fill_value`: NaN 대체 값
    * `margins`: 모든 데이터를 분석한 결과를 오른쪽과 아래에 붙일지 여부
    * `margins_name`: 마진 열(행)의 이름
    
만약 조건에 따른 데이터가 유일하게 선택되지 않으면 그룹연산을 하며 이 때 `aggfunc` 인수로 정의된 함수를 수행하여 대표값을 계산한다.

`pivot_table`를 메서드로 사용할 때는 객체 자체가 데이터가 되므로 `data` 인수가 필요하지 않다.

예를 들어 위에서 만들었던 피봇테이블은 `pivot_table` 명령으로 다음과 같이 만들 수도 있다. 인수의 순서에 주의하라.

In [6]:
import pandas as pd

data = {
    "도시": ["서울", "서울", "서울", "부산", "부산", "부산", "인천", "인천"],
    "연도": ["2015", "2010", "2005", "2015", "2010", "2005", "2015", "2010"],
    "인구": [9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 263203],
    "지역": ["수도권", "수도권", "수도권", "경상권", "경상권", "경상권", "수도권", "수도권"]
}
columns = ["도시", "연도", "인구", "지역"]
df1 = pd.DataFrame(data, columns=columns)
df1

도시    연도       인구   지역
0  서울  2015  9904312  수도권
1  서울  2010  9631482  수도권
2  서울  2005  9762546  수도권
3  부산  2015  3448737  경상권
4  부산  2010  3393191  경상권
5  부산  2005  3512547  경상권
6  인천  2015  2890451  수도권
7  인천  2010   263203  수도권

In [40]:
# pivot_table 에서는 pivot과는 차이점이 있다.

# pivot(행, 열, 데이터)
# pivot_table(데이터, 행, 열)

df1.pivot_table("인구", "도시", "연도")  # value : 인구, index(row) 도시, columns(col) 연도

연도       2005       2010       2015
도시                                 
부산  3512547.0  3393191.0  3448737.0
서울  9762546.0  9631482.0  9904312.0
인천        NaN   263203.0  2890451.0

`margins=True` 인수를 주면 `aggfunc`로 주어진 분석 방법을 해당 열의 모든 데이터, 해당 행의 모든 데이터 그리고 전체 데이터에 대해 적용한 결과를 같이 보여준다. **`aggfunc`가 주어지지 않았으면 평균을 계산한다.**

In [41]:
df1.pivot_table("인구", "도시", "연도", margins=True, margins_name="평균")  # aggfunc가 없으므로 디폴트로 평균이 계산된다.

연도       2005       2010       2015            평균
도시                                               
부산  3512547.0  3393191.0  3448737.0  3.451492e+06
서울  9762546.0  9631482.0  9904312.0  9.766113e+06
인천        NaN   263203.0  2890451.0  1.576827e+06
평균  6637546.5  4429292.0  5414500.0  5.350809e+06

이 결과에서 가장 오른쪽 평균 열의 첫번째 값 3451492은 모든 부산 인구 데이터의 평균, 두번째 값 9766113은 모든 서울 인구 데이터의 평균이다. 가장 아래의 합계 행의 첫번째 값은 2005년 데이터의 평균값, 두번째 값은 2010년 데이터의 평균값이다. 가장 오른쪽 아래의 값 5350809는 전체 데이터의 평균값이다. 다음 계산을 통해 이를 확인할 수 있다.

In [12]:
df1.pivot_table('인구', '도시', '연도', aggfunc='sum', margins = True, margins_name = '합계')

연도        2005        2010        2015        합계
도시                                              
부산   3512547.0   3393191.0   3448737.0  10354475
서울   9762546.0   9631482.0   9904312.0  29298340
인천         NaN    263203.0   2890451.0   3153654
합계  13275093.0  13287876.0  16243500.0  42806469

In [22]:
df1["인구"].mean()

5350808.625

행 인덱스나 열 인덱스에 리스트를 넣으면 다중 인덱스 테이블을 만든다.

In [42]:
df1.pivot_table("인구", index=["연도", "도시"])  # 다중 인덱스(행)

인구
연도   도시         
2005 부산  3512547
     서울  9762546
2010 부산  3393191
     서울  9631482
     인천   263203
2015 부산  3448737
     서울  9904312
     인천  2890451

### **3.1 TIP 데이터 예제**

식당에서 식사 후 내는 팁(tip)과 관련된 데이터를 이용하여 좀더 구체적으로 그룹분석 방법을 살펴본다. 우선 Seaborn 패키지에 설치된 샘플 데이터를 로드한다. 이 데이터프레임에서 각각의 컬럼은 다음을 뜻한다.

* total_bill: 식사대금
* tip: 팁
* sex: 성별
* smoker: 흡연/금연 여부
* day: 요일
* time: 시간
* size: 인원

In [44]:
import seaborn as sns

tips = sns.load_dataset("tips")
tips.tail()

total_bill   tip     sex smoker   day    time  size
239       29.03  5.92    Male     No   Sat  Dinner     3
240       27.18  2.00  Female    Yes   Sat  Dinner     2
241       22.67  2.00    Male    Yes   Sat  Dinner     2
242       17.82  1.75    Male     No   Sat  Dinner     2
243       18.78  3.00  Female     No  Thur  Dinner     2

분석의 목표는 식사 대금 대비 팁의 비율이 어떤 경우에 가장 높아지는지 찾는 것이다. 우선 식사대금와 팁의 비율을 나타내는 `tip_pct`를 추가하자.

In [45]:
tips['tip_pct'] = (tips['tip'] / tips['total_bill'] * 100).round(2)
tips.tail()

total_bill   tip     sex smoker   day    time  size  tip_pct
239       29.03  5.92    Male     No   Sat  Dinner     3    20.39
240       27.18  2.00  Female    Yes   Sat  Dinner     2     7.36
241       22.67  2.00    Male    Yes   Sat  Dinner     2     8.82
242       17.82  1.75    Male     No   Sat  Dinner     2     9.82
243       18.78  3.00  Female     No  Thur  Dinner     2    15.97

다음으로 각 열의 데이터에 대해 간단히 분포를 알아본다.

In [46]:
tips.describe()  # 기술통계량 확인

total_bill         tip        size     tip_pct
count  244.000000  244.000000  244.000000  244.000000
mean    19.785943    2.998279    2.569672   16.079754
std      8.902412    1.383638    0.951100    6.107020
min      3.070000    1.000000    1.000000    3.560000
25%     13.347500    2.000000    2.000000   12.910000
50%     17.795000    2.900000    2.000000   15.475000
75%     24.127500    3.562500    3.000000   19.147500
max     50.810000   10.000000    6.000000   71.030000

### **3.2 그룹별 통계**

우선 성별로 나누어 데이터 갯수를 세어본다.

In [47]:
tips.groupby("sex").count()

total_bill  tip  smoker  day  time  size  tip_pct
sex                                                      
Male           157  157     157  157   157   157      157
Female          87   87      87   87    87    87       87

데이터 갯수의 경우 NaN 데이터가 없다면 모두 같은 값이 나올 것이다. 이 때는 `size` 명령을 사용하면 더 간단히 표시된다. `size` 명령은 NaN이 있어도 상관하지 않는다.

In [28]:
tips.groupby("sex").size()

sex
Male      157
Female     87
dtype: int64

이번에는 성별과 흡연유무로 나누어 데이터의 갯수를 알아본다.

In [29]:
tips.groupby(["sex", "smoker"]).size()

sex     smoker
Male    Yes       60
        No        97
Female  Yes       33
        No        54
dtype: int64

좀 더 보기 좋도록 피봇 데이블 형태로 바꿀 수도 있다.

In [55]:
# pivot_table(데이터, 행, 열)에서 aggfunc를 쓰는 경우에
# data자리에 들어가는 데이터는 해당 데이터프레임에 존재하는 아무 변수를 넣어도 결과가 같다.
# 왜냐하면 aggfunc = 'count'로 성별, 흡연여부별 카운트(데이터 갯수 세기)를 하기 때문이다.

tips.pivot_table('tip', "sex", "smoker", aggfunc="count", margins=True)

smoker  Yes   No  All
sex                  
Male     60   97  157
Female   33   54   87
All      93  151  244

이제 성별과 흡연 여부에 따른 평균 팁 비율을 살펴본다.

In [56]:
tips.groupby("sex")[["tip_pct"]].mean()  # 성별 팁 비율 

tip_pct
sex              
Male    15.764713
Female  16.648276

In [57]:
tips.groupby("smoker")[["tip_pct"]].mean()  # 흡연여부별 팁 비율 

tip_pct
smoker           
Yes     16.319140
No      15.932318

`pivot_table` 명령을 사용할 수도 있다.

In [62]:
tips.pivot_table("tip_pct", "sex")  # 성별 팁 비율, 디폴트 값으로 평균 

tip_pct
sex              
Male    15.764713
Female  16.648276

In [63]:
tips.pivot_table("tip_pct", ["sex", "smoker"])  # 성별, 흡연여부별 팁 바율 (인덱스를 두개사용), 디폴트 값으로 평균 

tip_pct
sex    smoker           
Male   Yes     15.276667
       No      16.066598
Female Yes     18.214545
       No      15.691111

In [64]:
tips.pivot_table("tip_pct", "sex", "smoker")  # 성별, 흡연여부볼 팁 비율 (인덱스 1, 컬럼 1), 디폴트 값으로 평균

smoker        Yes         No
sex                         
Male    15.276667  16.066598
Female  18.214545  15.691111

여성 혹은 흡연자의 팁 비율이 높은 것을 볼 수 있다. 하지만 이 데이터에는 평균을 제외한 분산(variance) 등의 다른 통계값이 없으므로 `describe` 명령으로 여러가지 통계값을 한 번에 알아본다.

In [65]:
tips.groupby("sex")[["tip_pct"]].describe()  # 성별 팁 비율의 기술통계량

tip_pct                                                        
         count       mean       std   min    25%    50%     75%    max
sex                                                                   
Male     157.0  15.764713  6.477536  3.56  12.14  15.35  18.620  71.03
Female    87.0  16.648276  5.363127  5.64  14.04  15.56  19.425  41.67

In [66]:
tips.groupby("smoker")[["tip_pct"]].describe()  # 흡연여부볼 팁 비율의 기술통꼐량

tip_pct                                                       
         count       mean       std   min    25%    50%    75%    max
smoker                                                               
Yes       93.0  16.319140  8.511748  3.56  10.68  15.38  19.51  71.03
No       151.0  15.932318  3.990701  5.68  13.69  15.56  18.50  29.20

In [68]:
tips.groupby(["sex", "smoker"])[["tip_pct"]].describe()  # 성별, 흡연여부별 팁 비율의 기술통계량

tip_pct                                                         
                count       mean       std   min      25%    50%    75%    max
sex    smoker                                                                 
Male   Yes       60.0  15.276667  9.058490  3.56  10.1825  14.10  19.17  71.03
       No        97.0  16.066598  4.184634  7.18  13.1800  15.76  18.62  29.20
Female Yes       33.0  18.214545  7.159585  5.64  15.2400  17.39  19.82  41.67
       No        54.0  15.691111  3.641717  5.68  13.9700  14.97  18.16  25.27

### **연습 문제 7-3**

1. 팁의 비율이 요일과 점심/저녁 여부, 인원수에 어떤 영향을 받는지 살펴본다.
2. 어떤 요인이 가장 크게 작용하는지 판단할 수 있는 방법이 있는가?



In [73]:
import seaborn as sns

tips = sns.load_dataset("tips")
tips['tip_pct'] = (tips['tip'] / tips['total_bill'] * 100).round(2)  # 팁의 비율 
tips.tail()

total_bill   tip     sex smoker   day    time  size  tip_pct
239       29.03  5.92    Male     No   Sat  Dinner     3    20.39
240       27.18  2.00  Female    Yes   Sat  Dinner     2     7.36
241       22.67  2.00    Male    Yes   Sat  Dinner     2     8.82
242       17.82  1.75    Male     No   Sat  Dinner     2     9.82
243       18.78  3.00  Female     No  Thur  Dinner     2    15.97

In [75]:
tips.pivot_table('tip_pct', 'size', ['day', 'time'])

day        Thur            Fri                   Sat        Sun
time      Lunch Dinner   Lunch     Dinner     Dinner     Dinner
size                                                           
1     18.170000    NaN  22.380        NaN  23.180000        NaN
2     16.401277  15.97  18.196  16.267273  15.528679  18.086410
3     14.460000    NaN  18.770        NaN  15.142222  15.266000
4     14.552000    NaN     NaN  11.770000  13.828462  15.317222
5     12.140000    NaN     NaN        NaN  10.660000  15.986667
6     17.366667    NaN     NaN        NaN        NaN  10.380000

<hr>

이번에는 각 그룹에서 가장 많은 팁과 가장 적은 팁의 차이를 알아보자. 이 계산을 해 줄 수 있는 그룹연산 함수가 없으므로 함수를 직접 만들고 `agg` 메서드를 사용한다.

In [39]:
def peak_to_peak(x):
    return x.max() - x.min()


tips.groupby(["sex", "smoker"])[["tip"]].agg(peak_to_peak)

tip
sex    smoker      
Male   Yes     9.00
       No      7.75
Female Yes     5.50
       No      4.20

만약 여러가지 그룹연산을 동시에 하고 싶다면 다음과 같이 리스트를 이용한다.

In [40]:
tips.groupby(["sex", "smoker"]).agg(["mean", peak_to_peak])[["total_bill"]]

total_bill             
                    mean peak_to_peak
sex    smoker                        
Male   Yes     22.284500        43.56
       No      19.791237        40.82
Female Yes     17.977879        41.23
       No      18.105185        28.58

만약 데이터 열마다 다른 연산을 하고 싶다면 열 라벨과 연산 이름(또는 함수)를 딕셔너리로 넣는다.

In [41]:
tips.groupby(["sex", "smoker"]).agg(
    {'tip_pct': 'mean', 'total_bill': peak_to_peak})

tip_pct  total_bill
sex    smoker                      
Male   Yes     0.152771       43.56
       No      0.160669       40.82
Female Yes     0.182150       41.23
       No      0.156921       28.58

다음은 `pivot_table` 명령으로 더 복잡한 분석을 한 예이다.

In [42]:
tips.pivot_table(['tip_pct', 'size'], ['sex', 'day'], 'smoker')

size             tip_pct          
smoker            Yes        No       Yes        No
sex    day                                         
Male   Thur  2.300000  2.500000  0.164417  0.165706
       Fri   2.125000  2.000000  0.144730  0.138005
       Sat   2.629630  2.656250  0.139067  0.162132
       Sun   2.600000  2.883721  0.173964  0.158291
Female Thur  2.428571  2.480000  0.163073  0.155971
       Fri   2.000000  2.500000  0.209129  0.165296
       Sat   2.200000  2.307692  0.163817  0.147993
       Sun   2.500000  3.071429  0.237075  0.165710

In [43]:
tips.pivot_table('size', ['time', 'sex', 'smoker'], 'day',
                 aggfunc='sum', fill_value=0)

day                   Thur  Fri  Sat  Sun
time   sex    smoker                     
Lunch  Male   Yes       23    5    0    0
              No        50    0    0    0
       Female Yes       17    6    0    0
              No        60    3    0    0
Dinner Male   Yes        0   12   71   39
              No         0    4   85  124
       Female Yes        0    8   33   10
              No         2    2   30   43

### **연습 문제 7-4**

타이타닉 승객 데이터를 이용하여 다음 분석을 실시하라. 데이터는 다음과 같이 받을 수 있다.

```
titanic = sns.load_dataset("titanic")
```

1. `qcut` 명령으로 세 개의 나이 그룹을 만든다.
1. 성별, 선실, 나이 그룹에 의한 생존율을 데이터프레임으로 계산한다. 
행에는 성별 및 나이 그룹에 대한 다중 인덱스를 사용하고 열에는 선실 인덱스를 사용한다.
1. 성별 및 선실에 의한 생존율을 피봇 데이터 형태로 만든다.



In [76]:
import seaborn as sns

titanic = sns.load_dataset("titanic")
titanic.head()

survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True

In [170]:
# 1. qcut 명령으로 세 개의 나이 그룹을 만들어라.

titanic['age_groups'] = pd.qcut(titanic.age, 3, labels=['age1', 'age2', 'age3'])
titanic.age_groups.head(5)

0    age1
1    age3
2    age2
3    age3
4    age3
Name: age_groups, dtype: category
Categories (3, object): [age1 < age2 < age3]

In [136]:
# 2. 성별, 선실, 나이 그룹에 의한 생존율을 데이터프레임으로 계산한다. 
# 단, 행에는 성별 및 나이그룹에 대한 다중인덱스를 사용하고
# 열에는 선실 인덱스를 사용한다. 

# 방법 1. 우선, 변수 각각의 생존율을 계산해본 후 groupby 메서드를 이용해서 문제를 풀어본다.

# 1.1 성별 생존율 
titanic.groupby('sex')[['survived']].count() / titanic.survived.count()  

survived
sex             
female  0.352413
male    0.647587

In [137]:
# 1.2 선실별 생존율 

titanic.groupby('class')[['survived']].count() / titanic.survived.count()  

survived
class           
First   0.242424
Second  0.206510
Third   0.551066

In [138]:
# 1.3 나이그룹별 생존율 

titanic.groupby('age_groups')[['survived']].count() / titanic.survived.count()  

survived
age_groups          
age1        0.276094
age2        0.260382
age3        0.264871

In [169]:
# 1.4 성 및 나이그룹별(인덱스), 선실별(컬럼) 생존율 

# survived : 데이터
# mean() : pivot_table의 경우 디폴트 계산값이 평균이지만, groupby 메서드를 사용할 경우 mean() 메서드가 필요함. 
# groupby : sex(행1), age_groups(행2), class(행3)
# unstack으로 행3을 열1로 변환 

titanic['survived'].groupby([titanic["sex"], titanic['age_groups'], titanic['class']]).mean().unstack("class")

class                 First    Second     Third
sex    age_groups                              
female age1        0.954545  1.000000  0.508475
       age2        0.947368  0.909091  0.481481
       age3        0.977273  0.857143  0.250000
male   age1        0.500000  0.357143  0.158879
       age2        0.500000  0.076923  0.195652
       age3        0.347826  0.062500  0.055556

In [166]:
# 방법 2. pivot_table을 이용한 방법 
prob2 = titanic.pivot_table('survived', ['sex', 'age_groups'], 'class')  # 앞에서 차례로 데이터, 인덱스1, 인덱스2, 컬럼1
print(type(prob2))
prob2

<class 'pandas.core.frame.DataFrame'>


class                 First    Second     Third
sex    age_groups                              
female age1        0.954545  1.000000  0.508475
       age2        0.947368  0.909091  0.481481
       age3        0.977273  0.857143  0.250000
male   age1        0.500000  0.357143  0.158879
       age2        0.500000  0.076923  0.195652
       age3        0.347826  0.062500  0.055556

In [168]:
# 3. 성별 및 선실에 의한 생존율을 피봇 데이터 형태로 만들어라. 

prob3 = titanic.pivot_table('survived', 'sex', 'class')  # 데이터, 행1, 열1
print(type(prob3))
prob3

<class 'pandas.core.frame.DataFrame'>


class      First    Second     Third
sex                                 
female  0.968085  0.921053  0.500000
male    0.368852  0.157407  0.135447